In [0]:
spark

In [0]:
dbutils.fs.mount(
  source = "wasbs://markazapp-etl@markazapp.blob.core.windows.net/",
  mount_point = "/mnt/markazapp",
  extra_configs = {
 "fs.azure.account.key.markazapp.blob.core.windows.net": "Jns0vGUn9QkXbiJgtmaiWTNl7HpKWP+MCLrsSUkKtn/Oi2xOIUAw/ud3ZHiyt5+24tMkTCXKnA3E+AStX8SM1A=="
  })

True

In [0]:
%fs
ls "/mnt/markazapp"

path,name,size,modificationTime
dbfs:/mnt/markazapp/markazapp-raw/,markazapp-raw/,0,0
dbfs:/mnt/markazapp/markazapp-transformed/,markazapp-transformed/,0,0


In [0]:
transactions = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/markazapp/markazapp-raw/transactions.csv")
transactions.show(10)

+-----------+----------+-------------+--------+-------------+-----------+------+------------+
|CUSTOMER_ID|PRODUCT_ID|PURCHASE_DATE|QUANTITY|PRODUCT_PRICE|TOTAL_PRICE|RATING|      REVIEW|
+-----------+----------+-------------+--------+-------------+-----------+------+------------+
|          1|         1|   2022-01-01|       2|        200.0|      400.0|   1.0|äll was good|
|          2|         1|   2022-01-02|       1|        200.0|      200.0|   5.0|  heißen sie|
|          3|         1|   2022-01-03|       2|        200.0|      400.0|   3.5|   æ was bad|
|          4|         1|   2022-01-04|       3|        200.0|      600.0|   2.1|        çent|
|          5|         2|   2022-01-05|       1|        350.0|      350.0|   4.2|  heißen sie|
|          6|         2|   2022-01-06|       1|        350.0|      350.0|   2.1|   æ was bad|
|          7|         2|   2022-01-07|       1|        350.0|      350.0|   2.2|äll was good|
|          8|         2|   2022-01-08|       2|        350.0

In [0]:
from pyspark.sql import SparkSession
transactions.createOrReplaceTempView("transactions")

In [0]:
spark.sql("SELECT cast(purchase_date as date) FROM transactions").show(5)

+-------------+
|purchase_date|
+-------------+
|   2022-01-01|
|   2022-01-02|
|   2022-01-03|
|   2022-01-04|
|   2022-01-05|
+-------------+
only showing top 5 rows



In [0]:
%sql
select distinct product_id from transactions

product_id
1
6
5
9
4
8
7
10
2


In [0]:
from pyspark.sql.functions import col, sum, avg, count, expr, col, to_date

In [0]:
transactions.printSchema()

transactions.select("PRODUCT_ID").distinct().show()

root
 |-- CUSTOMER_ID: integer (nullable = true)
 |-- PRODUCT_ID: integer (nullable = true)
 |-- PURCHASE_DATE: date (nullable = true)
 |-- QUANTITY: integer (nullable = true)
 |-- PRODUCT_PRICE: double (nullable = true)
 |-- TOTAL_PRICE: double (nullable = true)
 |-- RATING: double (nullable = true)
 |-- REVIEW: string (nullable = true)

+----------+
|PRODUCT_ID|
+----------+
|         1|
|         6|
|         5|
|         9|
|         4|
|         8|
|         7|
|        10|
|         2|
+----------+



In [0]:
# Filter data for a specific product_id, e.g., product_id 1 with more than 2 order quantity
filtered_df = transactions.filter((transactions.PRODUCT_ID == 1) & (transactions.QUANTITY > 2))
filtered_df.show()

+-----------+----------+-------------+--------+-------------+-----------+------+------+
|CUSTOMER_ID|PRODUCT_ID|PURCHASE_DATE|QUANTITY|PRODUCT_PRICE|TOTAL_PRICE|RATING|REVIEW|
+-----------+----------+-------------+--------+-------------+-----------+------+------+
|          4|         1|   2022-01-04|       3|        200.0|      600.0|   2.1|  çent|
+-----------+----------+-------------+--------+-------------+-----------+------+------+



In [0]:
# Adding PRODUCT_CATEGORY column based on PRODUCT_ID value

transactions = transactions.withColumn(
    "PRODUCT_CATEGORY",
    expr("""
        CASE 
            WHEN PRODUCT_ID = 1 THEN 'Mobile phones'
            WHEN PRODUCT_ID = 2 THEN 'Game consoles'
            WHEN PRODUCT_ID = 3 THEN 'Household furniture'
            WHEN PRODUCT_ID = 4 THEN 'Home appliances'
            WHEN PRODUCT_ID = 5 THEN 'Clothing'
            WHEN PRODUCT_ID = 6 THEN 'Snow Sports'
            WHEN PRODUCT_ID = 7 THEN 'Camping & Hiking'
            WHEN PRODUCT_ID = 8 THEN 'Yard Games'
            WHEN PRODUCT_ID = 9 THEN 'Make-up'
            WHEN PRODUCT_ID = 10 THEN 'Beauty'
            ELSE 'Others'
        END
    """)
)

# Display the DataFrame with the new 'product_category' column
transactions.show(5)

+-----------+----------+-------------+--------+-------------+-----------+------+------------+----------------+
|CUSTOMER_ID|PRODUCT_ID|PURCHASE_DATE|QUANTITY|PRODUCT_PRICE|TOTAL_PRICE|RATING|      REVIEW|PRODUCT_CATEGORY|
+-----------+----------+-------------+--------+-------------+-----------+------+------------+----------------+
|          1|         1|   2022-01-01|       2|        200.0|      400.0|   1.0|äll was good|   Mobile phones|
|          2|         1|   2022-01-02|       1|        200.0|      200.0|   5.0|  heißen sie|   Mobile phones|
|          3|         1|   2022-01-03|       2|        200.0|      400.0|   3.5|   æ was bad|   Mobile phones|
|          4|         1|   2022-01-04|       3|        200.0|      600.0|   2.1|        çent|   Mobile phones|
|          5|         2|   2022-01-05|       1|        350.0|      350.0|   4.2|  heißen sie|   Game consoles|
+-----------+----------+-------------+--------+-------------+-----------+------+------------+----------------+
o

In [0]:
# convert purchase_date to date datatype
transactions = transactions.withColumn(
    "PURCHASE_DATE",
    to_date(col("PURCHASE_DATE"), "dd-MM-yyyy")
)

transactions.printSchema()

transactions.show(30)

root
 |-- CUSTOMER_ID: integer (nullable = true)
 |-- PRODUCT_ID: integer (nullable = true)
 |-- PURCHASE_DATE: date (nullable = true)
 |-- QUANTITY: integer (nullable = true)
 |-- PRODUCT_PRICE: double (nullable = true)
 |-- TOTAL_PRICE: double (nullable = true)
 |-- RATING: double (nullable = true)
 |-- REVIEW: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = false)

+-----------+----------+-------------+--------+-------------+-----------+------+------------+----------------+
|CUSTOMER_ID|PRODUCT_ID|PURCHASE_DATE|QUANTITY|PRODUCT_PRICE|TOTAL_PRICE|RATING|      REVIEW|PRODUCT_CATEGORY|
+-----------+----------+-------------+--------+-------------+-----------+------+------------+----------------+
|          1|         1|   2022-01-01|       2|        200.0|      400.0|   1.0|äll was good|   Mobile phones|
|          2|         1|   2022-01-02|       1|        200.0|      200.0|   5.0|  heißen sie|   Mobile phones|
|          3|         1|   2022-01-03|       2|        

In [0]:
# Calculate the total revenue for each product category
revenue_per_product = transactions.groupBy("PRODUCT_CATEGORY").agg(sum("TOTAL_PRICE").alias("TOTAL_REVENUE"))
revenue_per_product.show()

+----------------+-------------+
|PRODUCT_CATEGORY|TOTAL_REVENUE|
+----------------+-------------+
|Camping & Hiking|        399.0|
|         Make-up|       2700.0|
|   Game consoles|       1750.0|
|      Yard Games|        972.0|
|        Clothing|        500.0|
| Home appliances|       4050.0|
|   Mobile phones|       1600.0|
|     Snow Sports|        375.0|
|          Beauty|       3150.0|
+----------------+-------------+



In [0]:
# Count the number of purchases for each product category
total_orders_per_category = transactions.groupBy("PRODUCT_CATEGORY").agg(count("PURCHASE_DATE").alias("TOTAL_ORDERS"))
total_orders_per_category.show()

+----------------+------------+
|PRODUCT_CATEGORY|TOTAL_ORDERS|
+----------------+------------+
|Camping & Hiking|           1|
|         Make-up|           2|
|   Game consoles|           4|
|      Yard Games|           1|
|        Clothing|           5|
| Home appliances|           4|
|   Mobile phones|           4|
|     Snow Sports|           4|
|          Beauty|           1|
+----------------+------------+



In [0]:
# Statistics on RATING column
rating_stats = transactions.select("RATING").describe()
rating_stats.show()

+-------+------------------+
|summary|            RATING|
+-------+------------------+
|  count|                26|
|   mean| 2.988846153846154|
| stddev|0.7990510718249587|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



In [0]:
# Calculate the average review ratings for each product
average_ratings = transactions.groupBy("PRODUCT_CATEGORY").agg(
    avg(col("RATING")).alias("AVG_RATING"),
    count("*").alias("TOTAL_ORCERS")
)
average_ratings = average_ratings.withColumn("AVG_RATING", average_ratings["AVG_RATING"].cast("decimal(38,2)"))
average_ratings.show()

+----------------+----------+------------+
|PRODUCT_CATEGORY|AVG_RATING|TOTAL_ORCERS|
+----------------+----------+------------+
|Camping & Hiking|      3.02|           1|
|         Make-up|      3.03|           2|
|   Game consoles|      2.90|           4|
|      Yard Games|      3.03|           1|
|        Clothing|      3.00|           5|
| Home appliances|      3.09|           4|
|   Mobile phones|      2.90|           4|
|     Snow Sports|      3.01|           4|
|          Beauty|      2.99|           1|
+----------------+----------+------------+



In [0]:
# write transformed data to files
transactions.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/markazapp/markazapp-transformed/transactions")
revenue_per_product.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/markazapp/markazapp-transformed/revenue_per_product")
total_orders_per_category.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/markazapp/markazapp-transformed/total_orders_per_category")
rating_stats.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/markazapp/markazapp-transformed/rating_stats")
average_ratings.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/markazapp/markazapp-transformed/average_ratings")


In [0]:
dbutils.fs.unmount("/mnt/markazapp")

/mnt/markazapp has been unmounted.


True